# Explore NeRF Synthetic Blender Dataset (lego)

This notebook provides a quick sanity check for the NeRF Synthetic Blender **lego** scene:

- Load a few RGB images.
- Visualize camera pose distribution in 3D.

Make sure you have the dataset under `data/raw/nerf_synthetic/lego/` (or `data/raw/nerf_synthetic/lego/train/`) and a `transforms.json` or `transforms_train.json` file in `data/raw/nerf_synthetic/lego/`.


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import numpy as np
from PIL import Image

# Paths
PROJECT_ROOT = Path("..").resolve()
SCENE_DIR = PROJECT_ROOT / "data" / "raw" / "nerf_synthetic" / "lego"
print("Scene directory:", SCENE_DIR)



Scene directory: C:\Users\17320\Documents\Projects\Robust-NeRF\data\raw\lego


In [ ]:
# Show a few sample images
from typing import List

from pathlib import Path


def find_image_files(scene_dir: Path) -> List[Path]:
    exts = {".png", ".jpg", ".jpeg"}
    image_paths: List[Path] = []

    if scene_dir.exists():
        image_paths.extend(
            p for p in scene_dir.iterdir() if p.suffix.lower() in exts
        )

    train_dir = scene_dir / "train"
    if train_dir.exists():
        image_paths.extend(
            p for p in train_dir.iterdir() if p.suffix.lower() in exts
        )

    return sorted(image_paths)


image_files = find_image_files(SCENE_DIR)
print(f"Found {len(image_files)} images")

if not image_files:
    print("No images found. Check that the dataset is under data/raw/nerf_synthetic/lego/.")
else:
    num_samples = min(5, len(image_files))
    fig, axes = plt.subplots(1, num_samples, figsize=(3 * num_samples, 3))
    if num_samples == 1:
        axes = [axes]

    for ax, img_path in zip(axes, image_files[:num_samples]):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.set_title(img_path.name, fontsize=8)
        ax.axis("off")

    plt.tight_layout()
    plt.show()



Found 0 images
No images found. Check that the dataset is under data/raw/nerf_synthetic/lego/.


In [12]:
# Visualize camera pose cloud

candidates = [
    SCENE_DIR / "transforms.json",
    SCENE_DIR / "transforms_train.json",
]
transforms_path = None
for path in candidates:
    if path.exists():
        transforms_path = path
        break

if transforms_path is None:
    print("No transforms.json or transforms_train.json found in", SCENE_DIR)
else:
    print("Using transforms file:", transforms_path)
    with transforms_path.open("r") as f:
        transforms = json.load(f)

    positions = []
    for frame in transforms.get("frames", []):
        mat = np.array(frame["transform_matrix"], dtype=np.float32)
        positions.append(mat[:3, 3])

    if not positions:
        print("No frames found in transforms JSON.")
    else:
        positions = np.stack(positions, axis=0)

        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
        ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c="r", s=10)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.set_title("Camera centers (world space)")
        plt.tight_layout()
        plt.show()



No transforms.json or transforms_train.json found in C:\Users\17320\Documents\Projects\Robust-NeRF\data\raw\lego
